# 환경설정

In [107]:
import pandas as pd
# peak분석 라이브러리
from scipy.signal import find_peaks, peak_prominences   
import os  
import glob 
import re   

import math
import scipy
import numpy as np

from sklearn.ensemble import RandomForestClassifier   
from sklearn.naive_bayes import GaussianNB            
from sklearn.neighbors import KNeighborsClassifier    
from sklearn import svm                              
from sklearn.model_selection import cross_val_score, cross_val_predict    
from sklearn.metrics import classification_report, confusion_matrix   
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [35]:
# 경로 설정
os.chdir(r'C:\Users\joyh1\Desktop\motion-sense-master\data\A_DeviceMotion_data\A_DeviceMotion_data')

In [36]:
# 폴더 내 파일 리스트 불러오기 
file_list = glob.glob('*/*',recursive=True)
file_list

['dws_1\\sub_1.csv',
 'dws_1\\sub_10.csv',
 'dws_1\\sub_11.csv',
 'dws_1\\sub_12.csv',
 'dws_1\\sub_13.csv',
 'dws_1\\sub_14.csv',
 'dws_1\\sub_15.csv',
 'dws_1\\sub_16.csv',
 'dws_1\\sub_17.csv',
 'dws_1\\sub_18.csv',
 'dws_1\\sub_19.csv',
 'dws_1\\sub_2.csv',
 'dws_1\\sub_20.csv',
 'dws_1\\sub_21.csv',
 'dws_1\\sub_22.csv',
 'dws_1\\sub_23.csv',
 'dws_1\\sub_24.csv',
 'dws_1\\sub_3.csv',
 'dws_1\\sub_4.csv',
 'dws_1\\sub_5.csv',
 'dws_1\\sub_6.csv',
 'dws_1\\sub_7.csv',
 'dws_1\\sub_8.csv',
 'dws_1\\sub_9.csv',
 'dws_11\\sub_1.csv',
 'dws_11\\sub_10.csv',
 'dws_11\\sub_11.csv',
 'dws_11\\sub_12.csv',
 'dws_11\\sub_13.csv',
 'dws_11\\sub_14.csv',
 'dws_11\\sub_15.csv',
 'dws_11\\sub_16.csv',
 'dws_11\\sub_17.csv',
 'dws_11\\sub_18.csv',
 'dws_11\\sub_19.csv',
 'dws_11\\sub_2.csv',
 'dws_11\\sub_20.csv',
 'dws_11\\sub_21.csv',
 'dws_11\\sub_22.csv',
 'dws_11\\sub_23.csv',
 'dws_11\\sub_24.csv',
 'dws_11\\sub_3.csv',
 'dws_11\\sub_4.csv',
 'dws_11\\sub_5.csv',
 'dws_11\\sub_6.csv',
 'dw

In [37]:
# 파일명을 변수로 사용할 수 있도록 수정 
for f_name in file_list:
    temp = pd.read_csv(f_name)
    final_name_rev = f_name.replace('\\', '_')
    final_name_rev = final_name_rev.replace('.', '_') 
    globals()[final_name_rev]= temp

In [38]:
#mag함수 정의
def mag(df, column):
    df['mag'+str(column)] = np.sqrt(getattr(df,column+'.x')**2+getattr(df,column+'.y')**2+getattr(df,column+'.z')**2)    
    return df

# 모든 사람 행동 데이터

In [52]:
fls2 = [ fls_list for fls_list in fls if fls_list.endswith('_csv')]
HAR_total_df = []
for f in fls2:
    temp = globals()[f]
    HAR_total_df.append(temp.assign(exp_no = re.findall('\d+',f)[0]).assign(id = re.findall('\d+',f)[1]).assign(activity = f.split('_')[0]))
HAR_total = pd.concat(HAR_total_df)

In [53]:
# mag함수로 가속도, 회전율 새로운 칼럼 만들기
HAR_total = mag(HAR_total,'userAcceleration')
HAR_total = mag(HAR_total,"rotationRate")

In [54]:
HAR_total.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,exp_no,id,activity,maguserAcceleration,magrotationRate
0,0,1.528132,-0.733896,0.696372,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.294894,-0.184493,0.377542,1,1,dws,0.513360,1.370498
1,1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.219405,0.035846,0.114866,1,1,dws,0.250235,1.141648
2,2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.010714,0.134701,-0.167808,1,1,dws,0.215450,0.372530
3,3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,-0.008389,0.136788,0.094958,1,1,dws,0.166728,1.049628
4,4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.199441,0.353996,-0.044299,1,1,dws,0.408720,0.921229


In [55]:
del HAR_total['Unnamed: 0']
HAR_total['id'].value_counts()

19    71949
21    68932
16    65417
18    62808
3     62553
2     62339
1     62312
7     61276
8     60727
10    59796
15    59443
11    59292
6     57328
9     57136
14    56487
4     56066
22    56028
17    55539
20    55153
23    54556
12    53293
5     52283
13    51147
24    51005
Name: id, dtype: int64

In [56]:
del HAR_total['exp_no']

In [57]:
HAR_total.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,id,activity,maguserAcceleration,magrotationRate
0,1.528132,-0.733896,0.696372,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.294894,-0.184493,0.377542,1,dws,0.513360,1.370498
1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.219405,0.035846,0.114866,1,dws,0.250235,1.141648
2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.010714,0.134701,-0.167808,1,dws,0.215450,0.372530
3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,-0.008389,0.136788,0.094958,1,dws,0.166728,1.049628
4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.199441,0.353996,-0.044299,1,dws,0.408720,0.921229


In [59]:
# 기술 통계량 칼럼 추가하기 
aggs = {'maguserAcceleration':['mean', 'min', 'max','std','skew'],'magrotationRate':['mean', 'min', 'max','std','skew']}
HAR_summary = HAR_total.groupby(['id','activity']).agg(aggs)
HAR_summary

maguserAcceleration                                           \
                           mean       min       max       std       skew   
id activity                                                                
1  dws                 0.500533  0.015372  2.370524  0.274670   1.462860   
   jog                 1.410635  0.084504  6.419541  0.772475   1.354177   
   sit                 0.017672  0.001005  0.285272  0.015293   5.197562   
   std                 0.035460  0.001590  0.598686  0.030972   4.307244   
   ups                 0.399669  0.004309  2.507515  0.240441   1.554511   
...                         ...       ...       ...       ...        ...   
9  jog                 1.818199  0.053378  9.400228  0.984353   1.844488   
   sit                 0.020180  0.001667  0.867065  0.028510  12.965890   
   std                 0.040089  0.000806  0.504348  0.033102   2.970378   
   ups                 0.558511  0.027607  4.216134  0.373246   2.315180   
   wlk                 0.749008  0.024285  4.140044  0.479805   1.843309   

            magrotationRate                                            
                       mean       min        max       std       skew  
id activity                                                            
1  dws             2.098581  0.112195   7.138682  1.199613   0.673541  
   jog             4.173125  0.135332  12.645676  1.908202   0.160335  
   sit             0.044123  0.000364   0.926472  0.066348   4.942849  
   std             0.126757  0.000515   2.281784  0.140040   3.881329  
   ups             2.118329  0.109425  12.322634  1.282525   0.957503  
...                     ...       ...        ...       ...        ...  
9  jog             4.275354  0.224467  14.432254  2.061420   0.633833  
   sit             0.023110  0.000209   2.478304  0.080280  11.694253  
   std             0.115034  0.000661   1.606762  0.153745   3.277870  
   ups             2.168824  0.080533   8.671639  1.123480   0.569534  
   wlk             2.122859  0.121499   6.294985  0.967313   0.358224  

[144 rows x 10 columns]

In [60]:
HAR_summary = HAR_summary.reset_index()

In [66]:
HAR_summary['id'] = HAR_summary['id'].astype(int)
HAR_summary = HAR_summary.sort_values(by=['id'])
HAR_summary

id activity maguserAcceleration                                           \
                                mean       min       max       std       skew   
0     1      dws            0.500533  0.015372  2.370524  0.274670   1.462860   
1     1      jog            1.410635  0.084504  6.419541  0.772475   1.354177   
2     1      sit            0.017672  0.001005  0.285272  0.015293   5.197562   
3     1      std            0.035460  0.001590  0.598686  0.030972   4.307244   
4     1      ups            0.399669  0.004309  2.507515  0.240441   1.554511   
..   ..      ...                 ...       ...       ...       ...        ...   
97   24      jog            2.123950  0.082276  8.274447  1.314487   1.516866   
98   24      sit            0.009794  0.000375  0.467419  0.013282  14.360598   
99   24      std            0.033262  0.000984  0.516182  0.038588   4.053087   
100  24      ups            0.914871  0.025436  4.813813  0.502404   1.209217   
101  24      wlk            1.201580  0.071459  5.540921  0.742061   1.872870   

    magrotationRate                                           
               mean       min        max       std      skew  
0          2.098581  0.112195   7.138682  1.199613  0.673541  
1          4.173125  0.135332  12.645676  1.908202  0.160335  
2          0.044123  0.000364   0.926472  0.066348  4.942849  
3          0.126757  0.000515   2.281784  0.140040  3.881329  
4          2.118329  0.109425  12.322634  1.282525  0.957503  
..              ...       ...        ...       ...       ...  
97         5.087712  0.207078  20.387379  2.701979  0.820900  
98         0.025096  0.000340   1.041470  0.043211  8.991408  
99         0.197280  0.000735   2.331645  0.214542  2.719702  
100        3.196050  0.225433  10.282347  1.629103  0.723953  
101        2.565568  0.176692   9.680258  1.071932  0.628083  

[144 rows x 12 columns]

In [67]:
# user 개인정보 데이터 읽어오기 
subject_info = pd.read_csv('C:/Users/joyh1/Desktop/motion-sense-master/data/A_DeviceMotion_data/data_subjects_info.csv')

In [68]:
# subject_info랑 HAR_summary 병합하기
HAR_summary = pd.merge(HAR_summary,subject_info, how='left', left_on = 'id', right_on ='code')

c:\users\joyh1\anaconda3\envs\venvforpython\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [70]:
# 공통 칼럼 중 하나인 code 변수 지우기
del HAR_summary['code']

In [71]:
HAR_summary

,"(id, )","(activity, )","(maguserAcceleration, mean)","(maguserAcceleration, min)","(maguserAcceleration, max)","(maguserAcceleration, std)","(maguserAcceleration, skew)","(magrotationRate, mean)","(magrotationRate, min)","(magrotationRate, max)","(magrotationRate, std)","(magrotationRate, skew)",weight,height,age,gender
0,1,dws,0.500533,0.015372,2.370524,0.274670,1.462860,2.098581,0.112195,7.138682,1.199613,0.673541,102,188,46,1
1,1,jog,1.410635,0.084504,6.419541,0.772475,1.354177,4.173125,0.135332,12.645676,1.908202,0.160335,102,188,46,1
2,1,sit,0.017672,0.001005,0.285272,0.015293,5.197562,0.044123,0.000364,0.926472,0.066348,4.942849,102,188,46,1
3,1,std,0.035460,0.001590,0.598686,0.030972,4.307244,0.126757,0.000515,2.281784,0.140040,3.881329,102,188,46,1
4,1,ups,0.399669,0.004309,2.507515,0.240441,1.554511,2.118329,0.109425,12.322634,1.282525,0.957503,102,188,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,24,jog,2.123950,0.082276,8.274447,1.314487,1.516866,5.087712,0.207078,20.387379,2.701979,0.820900,74,173,18,0
140,24,sit,0.009794,0.000375,0.467419,0.013282,14.360598,0.025096,0.000340,1.041470,0.043211,8.991408,74,173,18,0
141,24,std,0.033262,0.000984,0.516182,0.038588,4.053087,0.197280,0.000735,2.331645,0.214542,2.719702,74,173,18,0
142,24,ups,0.914871,0.025436,4.813813,0.502404,1.209217,3.196050,0.225433,10.282347,1.629103,0.723953,74,173,18,0


In [72]:
# 변수명 추출
HAR_summary_names = HAR_summary.columns
HAR_summary_names = [list(i) for i in HAR_summary_names]
HAR_summary_names = [''.join(i) for i in HAR_summary_names]
HAR_summary.columns  = HAR_summary_names
HAR_summary

,id,activity,maguserAccelerationmean,maguserAccelerationmin,maguserAccelerationmax,maguserAccelerationstd,maguserAccelerationskew,magrotationRatemean,magrotationRatemin,magrotationRatemax,magrotationRatestd,magrotationRateskew,weight,height,age,gender
0,1,dws,0.500533,0.015372,2.370524,0.274670,1.462860,2.098581,0.112195,7.138682,1.199613,0.673541,102,188,46,1
1,1,jog,1.410635,0.084504,6.419541,0.772475,1.354177,4.173125,0.135332,12.645676,1.908202,0.160335,102,188,46,1
2,1,sit,0.017672,0.001005,0.285272,0.015293,5.197562,0.044123,0.000364,0.926472,0.066348,4.942849,102,188,46,1
3,1,std,0.035460,0.001590,0.598686,0.030972,4.307244,0.126757,0.000515,2.281784,0.140040,3.881329,102,188,46,1
4,1,ups,0.399669,0.004309,2.507515,0.240441,1.554511,2.118329,0.109425,12.322634,1.282525,0.957503,102,188,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,24,jog,2.123950,0.082276,8.274447,1.314487,1.516866,5.087712,0.207078,20.387379,2.701979,0.820900,74,173,18,0
140,24,sit,0.009794,0.000375,0.467419,0.013282,14.360598,0.025096,0.000340,1.041470,0.043211,8.991408,74,173,18,0
141,24,std,0.033262,0.000984,0.516182,0.038588,4.053087,0.197280,0.000735,2.331645,0.214542,2.719702,74,173,18,0
142,24,ups,0.914871,0.025436,4.813813,0.502404,1.209217,3.196050,0.225433,10.282347,1.629103,0.723953,74,173,18,0


# Train, Test 데이터 나누기

In [96]:
# train, test 나누기
train_d = HAR_summary.loc[(HAR_summary['id'] > 0) & (HAR_summary['id'] < 20)]
test_d = HAR_summary.loc[(HAR_summary['id'] > 19) & (HAR_summary['id'] < 25)]
print('train 데이터 :', train_d.shape)
print('test 데아터 :', test_d.shape)

train 데이터 : (114, 16)
test 데아터 : (30, 16)


In [98]:
train_d

,id,activity,maguserAccelerationmean,maguserAccelerationmin,maguserAccelerationmax,maguserAccelerationstd,maguserAccelerationskew,magrotationRatemean,magrotationRatemin,magrotationRatemax,magrotationRatestd,magrotationRateskew,weight,height,age,gender
0,1,dws,0.500533,0.015372,2.370524,0.274670,1.462860,2.098581,0.112195,7.138682,1.199613,0.673541,102,188,46,1
1,1,jog,1.410635,0.084504,6.419541,0.772475,1.354177,4.173125,0.135332,12.645676,1.908202,0.160335,102,188,46,1
2,1,sit,0.017672,0.001005,0.285272,0.015293,5.197562,0.044123,0.000364,0.926472,0.066348,4.942849,102,188,46,1
3,1,std,0.035460,0.001590,0.598686,0.030972,4.307244,0.126757,0.000515,2.281784,0.140040,3.881329,102,188,46,1
4,1,ups,0.399669,0.004309,2.507515,0.240441,1.554511,2.118329,0.109425,12.322634,1.282525,0.957503,102,188,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,19,dws,0.517353,0.011874,2.698336,0.352194,2.143703,1.146666,0.046912,4.600473,0.685838,1.373940,78,164,28,0
110,19,sit,0.094520,0.000820,0.241429,0.056502,-0.003722,0.055082,0.000471,1.091974,0.079026,4.670839,78,164,28,0
111,19,std,0.075072,0.001340,0.306569,0.036582,-0.228335,0.075327,0.001374,0.908275,0.090617,3.469247,78,164,28,0
112,19,ups,0.297682,0.009076,1.158621,0.149616,1.327806,0.907087,0.049769,3.223361,0.512071,1.411719,78,164,28,0


In [100]:
# train, test 각 데이터에서 mag 또는 activity 문자열이 있는 변수 추출 
train_activity = train_d[[i for i in train_d.columns  if any(re.findall('mag|activity', i))]]
test_activity = test_d[[i for i in test_d.columns  if any(re.findall('mag|activity', i))]]
train_activity.head()

,activity,maguserAccelerationmean,maguserAccelerationmin,maguserAccelerationmax,maguserAccelerationstd,maguserAccelerationskew,magrotationRatemean,magrotationRatemin,magrotationRatemax,magrotationRatestd,magrotationRateskew
0,dws,0.500533,0.015372,2.370524,0.274670,1.462860,2.098581,0.112195,7.138682,1.199613,0.673541
1,jog,1.410635,0.084504,6.419541,0.772475,1.354177,4.173125,0.135332,12.645676,1.908202,0.160335
2,sit,0.017672,0.001005,0.285272,0.015293,5.197562,0.044123,0.000364,0.926472,0.066348,4.942849
3,std,0.035460,0.001590,0.598686,0.030972,4.307244,0.126757,0.000515,2.281784,0.140040,3.881329
4,ups,0.399669,0.004309,2.507515,0.240441,1.554511,2.118329,0.109425,12.322634,1.282525,0.957503


# 모델 적용

In [121]:
# train, test 각 데이터에 feature와 label 분할해주기
train_x = train_activity.drop(['activity'],axis=1)
train_y = train_activity.activity

test_x = test_activity.drop(['activity'],axis=1)
test_y = test_activity.activity

# 모델 학습 
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
rf.fit(train_x, train_y)

# test데이터 이용해 예측
pred = rf.predict(test_x)
# 예측값과 실제값(test_y)와 비교
accuracy = accuracy_score(test_y, pred)

# oobscore : 예측이 얼마나 정확한가에 대한 수치값
print('oob_score :', rf.oob_score_)
print('Mean Accuracy score:', accuracy)
print()
print('-----------------------Confusion Matrix-------------------------')
target_names = list(set(train_y))
print(classification_report(test_y, pred, target_names=target_names))

oob_score : 0.8070175438596491
Mean Accuracy score: 0.7

-----------------------Confusion Matrix-------------------------
              precision    recall  f1-score   support

         jog       0.60      0.60      0.60         5
         ups       0.71      1.00      0.83         5
         sit       0.71      1.00      0.83         5
         dws       1.00      0.60      0.75         5
         std       1.00      0.40      0.57         5
         wlk       0.50      0.60      0.55         5

    accuracy                           0.70        30
   macro avg       0.75      0.70      0.69        30
weighted avg       0.75      0.70      0.69        30

